In [3]:
!pip install datasets
!pip install scikit-learn

from transformers import BertTokenizer, BertForQuestionAnswering, AdamW
from datasets import load_dataset
import torch
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score

# Load SQuAD 1.1 dataset
squad = load_dataset("squad")

# Tokenizer and model
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForQuestionAnswering.from_pretrained("bert-base-uncased")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 20.0 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.4/39.4 MB 28.8 MB/s eta 0:00:0000:0100:01


README.md:   0%|          | 0.00/7.62k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/1.82M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
# Preprocessing function
def preprocess(example):
    inputs = tokenizer(
        example["question"], example["context"], truncation=True, padding="max_length", max_length=384
    )
    inputs["start_positions"] = example["answers"]["answer_start"][0]
    inputs["end_positions"] = example["answers"]["answer_start"][0] + len(example["answers"]["text"][0]) - 1
    return inputs

In [5]:
# Apply preprocessing
train_data = squad["train"].map(preprocess, batched=True)
val_data = squad["validation"].map(preprocess, batched=True)


Map:   0%|          | 0/87599 [00:00<?, ? examples/s]

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

TypeError: list indices must be integers or slices, not str

In [ ]:
# Dataloader
train_loader = DataLoader(train_data, batch_size=16, shuffle=True)
val_loader = DataLoader(val_data, batch_size=16)


In [ ]:
# Optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

# Training loop
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

In [ ]:
for epoch in range(3):  
    model.train()
    for batch in train_loader:
        inputs = {key: torch.tensor(val).to(device) for key, val in batch.items() if key in ["input_ids", "attention_mask"]}
        labels = {key: torch.tensor(val).to(device) for key, val in batch.items() if key in ["start_positions", "end_positions"]}
        optimizer.zero_grad()
        outputs = model(**inputs, **labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()


In [ ]:
# Evaluation loop
model.eval()
exact_matches = []
for batch in val_loader:
    with torch.no_grad():
        inputs = {key: torch.tensor(val).to(device) for key, val in batch.items() if key in ["input_ids", "attention_mask"]}
        labels = batch["answers"]  # Extract ground-truth answers
        outputs = model(**inputs)
        
        # Get predicted start and end logits
        start_logits = outputs.start_logits
        end_logits = outputs.end_logits
        
        # Get the predicted start and end positions
        start_positions = torch.argmax(start_logits, dim=1).tolist()
        end_positions = torch.argmax(end_logits, dim=1).tolist()
        
        # Convert predictions to text spans
        input_ids = inputs["input_ids"].tolist()
        predicted_answers = [
            tokenizer.decode(input_id[start:end + 1], skip_special_tokens=True)
            for input_id, start, end in zip(input_ids, start_positions, end_positions)
        ]
        
        # Normalize predictions and ground truths
        for prediction, ground_truths in zip(predicted_answers, labels):
            ground_truth_texts = [gt_text.strip() for gt_text in ground_truths["text"]]
            exact_matches.append(
                any(normalize_text(prediction) == normalize_text(gt) for gt in ground_truth_texts)
            )

# Calculate and print EM score
em_score = sum(exact_matches) / len(exact_matches) * 100
print(f"Exact Match (EM) Score: {em_score:.2f}%")
